In [179]:
import numpy as np
from scipy.sparse import random
from scipy import stats
from ddeint import ddeint
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.linalg import qr

In [2]:
# Reservoir generation 
def gen_matrix(shape, sparsity, sd=1, mean=0, loc_seed=100, val_seed=100, pdf="gaussian", seeded=True):
    
    def seeded_rvs_gauss(array_len):
            return stats.norm(loc=mean, scale=sd).rvs(random_state = val_seed, size=array_len)

    def seeded_rvs_uniform(array_len):
        return stats.uniform(loc=mean, scale=sd).rvs(random_state = val_seed, size=array_len)

    m = shape[0]
    n = shape[1]

    if seeded == True:
        
        if pdf == "gaussian":
            M = random(m, n, density=sparsity, random_state=loc_seed, data_rvs=seeded_rvs_gauss).A
            return M

        if pdf == "uniform":
            M = random(m, n, density=sparsity, random_state=loc_seed, data_rvs=seeded_rvs_uniform).A
            return M

        if pdf == "ones":
            M = random(m, n, density = sparsity, random_state=loc_seed, data_rvs=np.ones).A
            return M
        else: 
            print("No such pdf")
            
    elif seeded == False:
        
        if pdf == "gaussian":
            unseeded_rvs = stats.norm(loc=mean, scale=sd).rvs
            M = random(m, n, density=sparsity, data_rvs=unseeded_rvs).A
            return M

        if pdf == "uniform":
            unseeded_rvs = stats.uniform(loc=mean, scale=sd).rvs
            M = random(m, n, density=sparsity, data_rvs=unseeded_rvs).A
            return M

        if pdf == "ones":
            M = random(m, n, density = sparsity, data_rvs=np.ones).A
            return M
        else: 
            print("No such pdf")
            
    else:
        print("Seeded was neither true nor false")

def spectral_radius(mat):
    max_abs_eigenvalue = -1
    eigenvalues, eigenvectors = np.linalg.eig(mat)
    for eigenvalue in eigenvalues:
        if abs(eigenvalue) > max_abs_eigenvalue:
            max_abs_eigenvalue = abs(eigenvalue)
    return max_abs_eigenvalue

def spectral_radius_matrix(mat, desired_spec_rad):
    M_sr = spectral_radius(mat)
    if M_sr == 0:
        return mat
    else:
        mat = mat*(desired_spec_rad/M_sr)
        return mat
    
# ESN equations
def sigma(value):
    return np.tanh(value)

def state(x_prev, z_curr, A, gamma, C, s, zeta):
    z_curr = np.atleast_2d(z_curr)
    x_curr = sigma(np.matmul(A, x_prev) + gamma*np.matmul(C, z_curr) + s*zeta)
    return x_curr

def observation(x_curr, w):
    z_next = np.matmul(np.transpose(w), x_curr)
    return z_next

# Training equations
def listening(training_data, x_0, A, gamma, C, s, zeta):
    state_dict = {'all_states': None,
                  'best_weight': None, 
                  'input_data': None}
    
    T = len(training_data)
    
    for t in range(1, T+1):
        if t == 1:
            x_curr = x_0
            X = np.array(x_curr)
            z_curr = training_data.loc[t]['tau17']
            Z = np.atleast_2d(np.array([z_curr]))
        else:
            x_curr = state(x_curr, z_curr, A, gamma, C, s, zeta)
            X = np.column_stack((X, x_curr))
            z_curr = training_data.loc[t]['tau17']
            Z = np.column_stack((Z, z_curr))
            
    state_dict['last_state'] = x_curr
    state_dict['all_states'] = X
    state_dict['input_data'] = Z
    
    return state_dict
    
def regression_sol(ld, state_dict, N):
    X = state_dict['all_states']
    Z = state_dict['input_data']
    
    X = X[:, 1001:]
    Z = Z[:, 1001:]

    X_transpose = X.transpose()
    Z_transpose = Z.transpose()
    XZ_transpose = np.matmul(X, Z_transpose)
    
    inverse_term = np.linalg.inv(np.matmul(X, X_transpose) + ld*np.identity(N))
    W_best = np.matmul(inverse_term, XZ_transpose)
    
    return W_best

def prediction(state_dict, weight, testing_data, training_data, A, gamma, C, s, zeta, platt_err):
    prediction_dict = {'testing_error': None,
                       'z_actuals': None,
                       'z_predictions': None}
    
    z_actuals = []
    z_predictions = []
    
    last_state = state_dict.get('last_state')
    testing_error = 0
    
    T = len(training_data)
    T_bar = len(testing_data)
    
    weight_sum = 0
    x_prev = last_state
    for t_bar in range(T+1, T+T_bar+1):
        z_predict = observation(x_prev, weight)[0][0]
        x_prev = state(x_prev, z_predict, A, gamma, C, s, zeta)
        z_predictions.append(z_predict)
        
        z_actual = testing_data.loc[t_bar]['tau17']
        z_actuals.append(z_actual)
        
        if platt_err == True:
            testing_error = testing_error + ((z_predict - z_actual)**2)*np.exp(-((t_bar-T)/T_bar))
            weight_sum = weight_sum + np.exp(-((t_bar-T)/T_bar))
        else:    
            testing_error = testing_error + (z_predict - z_actual)**2
            weight_sum = weight_sum + 1
    
    prediction_dict['testing_error'] = testing_error/weight_sum
    prediction_dict['z_actuals'] = z_actuals
    prediction_dict['z_predictions'] = z_predictions        
        
    return prediction_dict

def training_error(state_dict, weight, training_data):
    training_error_dict = {'training_error': None,
                           'z_actuals': None,
                           'z_predictions': None}
    
    z_actuals = []
    z_predictions = []
    
    X = state_dict.get('all_states')
    
    X = X[:, 1001:]
    
    training_error = 0
    
    T = len(training_data)
    
    for t in range(1001, T):
        x_prev = X[:, t-1001]
        z_predict = observation(x_prev, weight)[0]
        
        z_predictions.append(z_predict)
        z_actual = training_data.loc[t]['tau17']
        z_actuals.append(z_actual)
        training_error = training_error + (z_predict - z_actual)**2
        
    training_error = training_error/(T-1000)
    
    training_error_dict['training_error'] = training_error
    training_error_dict['z_actuals'] = z_actuals
    training_error_dict['z_predictions'] = z_predictions
        
    return training_error_dict

# data generation functions
def generate_MG_data(init_val, tau, a, b, n):
    def MackeyGlass(z, t):
        return (a * z(t - tau)) / (1 + z(t - tau)**n) - b*z(t)

    def initial(t):
        return init_val

    time_space = np.arange(0, 5000, 1)

    z_solution_new = ddeint(MackeyGlass, initial, time_space)
    z_solution_new  = [ data_pt[0] for data_pt in z_solution_new[1:] ]
    z_solution_new.insert(0, init_val)
    z_solution_new = [ np.tanh(unshifted_val-1) for unshifted_val in z_solution_new ]
    z_sol_new_df = pd.DataFrame(z_solution_new, columns=['tau17'])

    z_sol_new_df.index = z_sol_new_df.index + 1
    training_data = z_sol_new_df.loc[1:3001]
    testing_data = z_sol_new_df.loc[3002:]
    
    return training_data, testing_data

# miscellaneous plotting functions
def state_plot(state_dict, plotwith_init, node=0):
    X = state_dict.get('all_states')
    if plotwith_init == True:
        state_plot, state_ax = plt.subplots(figsize=(20,5))
        state_ax.plot(X[node][:])
        state_ax.set_title('Plot of States at node {}'.format(node))
        state_ax.set_xlabel('time')
        state_ax.set_ylabel('state of node {}'.format(node))
        
        return (np.amin(X[node][:]), np.amax(X[node][:]))
    
    if plotwith_init == False:
        state_plot, state_ax = plt.subplots(figsize=(20,5))
        state_ax.plot(X[node][1001:])
        state_ax.set_title('Plot of States at node {}'.format(node))
        state_ax.set_xlabel('time')
        state_ax.set_ylabel('state of node {}'.format(node))
    
        return (np.amin(X[node][1001:]), np.amax(X[node][1001:]))
                
def hist_accuracy_plot(actuals, predictions, with_bars=False):
    if with_bars == False:
        sns.kdeplot(actuals, label='actual', shade=True, color='red')
        sns.kdeplot(predictions, label='prediction', shade=True, color='skyblue')
        plt.legend()
        
    if with_bars == True:
        sns.histplot(actuals, label='actual', color='red', kde=True)
        sns.histplot(predictions, label='prediction', color='skyblue', kde=True)
        plt.legend()

In [56]:
data = generate_MG_data(1.2, 17, 0.2, 0.1, 10)

warmup_data = data[0]

/Users/hannah/Library/Python/3.8/lib/python/site-packages/ddeint/ddeint.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([g(tt[0])] + results)


In [186]:
N = 1000
ld = 10**(-14)
C = gen_matrix(shape=(N,1), sparsity=0.6, sd=4, mean=-2, pdf="uniform", seeded=True, loc_seed=407, val_seed=408)
gamma = 0.8
A = gen_matrix(shape=(N,N), sparsity=0.01, sd=2, mean=-1, pdf="uniform", seeded=True, loc_seed=500, val_seed=508)
spec_rad = 1.2
A = spectral_radius_matrix(A, spec_rad)
s = 0.2
zeta = gen_matrix(shape=(N,1), sparsity=0.6, pdf="ones", loc_seed=400, seeded=True)
x_0 = np.zeros(shape=(N,1), dtype=float)

state_dict = listening(warmup_data, x_0, A, gamma, C, s, zeta)
weights = regression_sol(ld, state_dict, N) 

In [190]:
def pf0_px(x): # (N,1) vector
    return csc_matrix(weights.transpose())

def outer_term(x, z):
    csc_A = csc_matrix(A)
    csc_C = csc_matrix(C)
    csc_zeta = csc_matrix(zeta)
    
    tanh_term = np.tanh(csc_A*x + gamma*csc_C*z + s*csc_zeta)
    
    return csc_matrix(1 - np.power(tanh_term, 2))

def pf_px(x, z): # (N,1) vector and scalar
    outer = outer_term(x, z)
    return outer.multiply(A)
    
def pf_pz(x, z): # (N,1) vector and scalar
    outer = outer_term(x, z)
    return outer.multiply(C)

def gram_schmidt(u):
    alpha = np.zeros(shape=(N,))
    for j in range(N):
        for k in range(j):
            u[: , j] = u[: , j] - np.dot(u[: , k], u[: , j]) * u[: , k]
        alpha[j] = np.linalg.norm(u[: , j])
        u[:, j] = u[: , j] / alpha[j]
    return u, alpha

In [192]:
T = 100
T_norm = 5
delta = 10**(-7)
h = 1   # needs to correspond to the step size in the data set

In [194]:
Delta = delta * np.identity(N)

Lambda = np.zeros(shape=(N, ))
Lambda_prev = np.zeros(shape=(N, ))

x_prev = state_dict['last_state']
z_curr = state_dict['input_data'][0, -1]

J0 = pf0_px(x_prev)

for t in range(0, T):
    
    x_curr = state(x_prev, z_curr, A, gamma, C, s, zeta)
    z_curr = observation(x_curr, weights) 
    x_prev = x_curr
    
    J1 = pf_px(x_curr, z_curr) 
    J2 = pf_pz(x_curr, z_curr)
    J = J1 + J2*J0
    Delta = J*Delta
    
    Delta, alpha = gram_schmidt(Delta)
    Lambda = Lambda + np.log(alpha)

    Lambda_prev = Lambda

Lambda = Lambda/(T*h)

KeyboardInterrupt: 

In [ ]:
# Kaplan-Yorke dimension

sum_Lambda = 0

D_KY = 0

for Lambda_id in range(0, len(Lambda)):
    sum_Lambda = sum_Lambda + Lambda[Lambda_id]
    if sum_Lambda < 0:
        D_KY = Lambda_id - 1 - ((sum_Lambda - Lambda[Lambda_id])/Lambda[Lambda_id])
        
D_KY = D_KY + 1